World Coins: A collection of coin images from 32 different currencies.

In [ ]:
# Seznam knihoven
import os
import json
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
from itertools import combinations
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input as preprocess_resnet
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input as preprocess_mobilenet
from tensorflow.keras.applications.efficientnet import preprocess_input as preprocess_efficientnet
from tabulate import tabulate

Vyhodnocení logů z trénování

In [5]:
import pandas as pd
from tabulate import tabulate

# Cesty k souborům
path_efficientnet = "models/training_logs/EfficientNetB3.csv"
path_mobilenet = "models/training_logs/MobileNetV3.csv"
path_resnet = "models/training_logs/ResNet50.csv"

file_paths = {
    "EfficientNetB3": path_efficientnet,
    "MobileNetV3": path_mobilenet,
    "ResNet50": path_resnet
}

# Připravení tabulky pouze s požadovanými sloupci
best_results = []

for model_name, file_path in file_paths.items():
    df = pd.read_csv(file_path)
    
    best_row = df.loc[df["val_loss"].idxmin()]
    best_results.append({
        "Model": model_name,
        "Loss": best_row["loss"],
        "Accuracy": best_row["accuracy"],
        "Val Loss": best_row["val_loss"],
        "Val Accuracy": best_row["val_accuracy"]
    })

# Vytvoření tabulky s výsledky
df_results = pd.DataFrame(best_results)
print(tabulate(df_results, headers='keys', tablefmt='grid'))

+----+----------------+----------+------------+------------+----------------+
|    | Model          |     Loss |   Accuracy |   Val Loss |   Val Accuracy |
+====+================+==========+============+============+================+
|  0 | EfficientNetB3 | 0.233056 |   0.936022 |    1.01966 |       0.781991 |
+----+----------------+----------+------------+------------+----------------+
|  1 | MobileNetV3    | 0.190648 |   0.952867 |    1.00535 |       0.78436  |
+----+----------------+----------+------------+------------+----------------+
|  2 | ResNet50       | 0.174244 |   0.942663 |    1.07016 |       0.791469 |
+----+----------------+----------+------------+------------+----------------+


Porovnání modelů na testovacích datech

In [ ]:
# Cesty k modelům a testovacím datům
MODEL_PATHS = {
    "ResNet50": ("models/best_ResNet50.keras", preprocess_resnet),
    "MobileNetV3": ("models/best_MobileNetV3.keras", preprocess_mobilenet),
    "EfficientNetB3": ("models/best_EfficientNetB3.keras", preprocess_efficientnet)
}
TEST_DATA_PATH = "data/coins/data/test/"
JSON_PATH = "data/cat_to_name.json"

# Načtení mapování kategorií
with open(JSON_PATH, "r") as f:
    cat_to_name = json.load(f)

# Funkce pro vytvoření datového generátoru s odpovídajícím předzpracováním
def create_generator(preprocess_function, augmentation=False):
    if augmentation:
        datagen = ImageDataGenerator(
            preprocessing_function=preprocess_function,
            rotation_range=30,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range=[0.8, 1.2],
            channel_shift_range=20.0
        )
    else:
        datagen = ImageDataGenerator(preprocessing_function=preprocess_function)
    
    return datagen.flow_from_directory(
        TEST_DATA_PATH,
        target_size=(224, 224),
        batch_size=32,
        class_mode='sparse',
        shuffle=False
    )

# Načtení modelů a odpovídajících generátorů
test_generators = {}
models = {}
for model_name, (model_path, preprocess_function) in MODEL_PATHS.items():
    models[model_name] = tf.keras.models.load_model(model_path)
    test_generators[model_name] = create_generator(preprocess_function)

# Funkce pro vyhodnocení přesnosti
def evaluate_model(predictions, true_classes):
    predicted_classes = np.argmax(predictions, axis=1)
    correct = np.sum(predicted_classes == true_classes)
    total = len(true_classes)
    accuracy = 100 * correct / total
    return total, correct, accuracy

# Uchování výsledků
results = []

# Vyhodnocení jednotlivých modelů
predictions_cache = {}
for model_name, model in models.items():
    test_generator = test_generators[model_name]
    predictions = model.predict(test_generator)
    predictions_cache[model_name] = predictions
    total, correct, accuracy = evaluate_model(predictions, test_generator.classes)
    results.append({"Model": model_name, "Testované soubory": total, "Úspěšné klasifikace": correct, "Úspěšnost (%)": accuracy})

# Vyhodnocení kombinací modelů
for r in range(2, len(MODEL_PATHS) + 1):
    for combo in combinations(MODEL_PATHS.keys(), r):
        combo_name = " + ".join(combo)
        ensemble_predictions = np.mean([predictions_cache[m] for m in combo], axis=0)
        total, correct, accuracy = evaluate_model(ensemble_predictions, test_generator.classes)
        results.append({"Model": combo_name, "Testované soubory": total, "Úspěšné klasifikace": correct, "Úspěšnost (%)": accuracy})

# Výstupní DataFrame
df = pd.DataFrame(results)
print(tabulate(df, headers='keys', tablefmt='grid'))

Found 844 images belonging to 211 classes.
Found 844 images belonging to 211 classes.
Found 844 images belonging to 211 classes.


c:\Users\MilanPštross\Documents\Word_Coins\.venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


27/27 ━━━━━━━━━━━━━━━━━━━━ 36s 1s/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 35s 1s/step
27/27 ━━━━━━━━━━━━━━━━━━━━ 30s 1s/step
+----+-----------------------------------------+---------------------+-----------------------+-----------------+
|    | Model                                   |   Testované soubory |   Úspěšné klasifikace |   Úspěšnost (%) |
+====+=========================================+=====================+=======================+=================+
|  0 | ResNet50                                |                 844 |                   682 |         80.8057 |
+----+-----------------------------------------+---------------------+-----------------------+-----------------+
|  1 | MobileNetV3                             |                 844 |                   667 |         79.0284 |
+----+-----------------------------------------+---------------------+-----------------------+-----------------+
|  2 | EfficientNetB3                          |                 844 |                   662

Napojení na akturální kurzy ČNB

In [3]:
# Cesty k datům
MODEL_PATH = "models/best_ResNet50.keras"
TEST_DATA_PATH = "data/coins/data/test/"
TRAIN_DATA_PATH = "data/coins/data/train/"
JSON_PATH = "models/cat_to_name/cat_to_name_mod.json"

# Načtení mapování kategorií
with open(JSON_PATH, "r") as f:
    cat_to_name = json.load(f)

# Načtení modelu
model = tf.keras.models.load_model(MODEL_PATH)

# Načtení mapování tříd z trénovacího datasetu
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_resnet)
train_generator = train_datagen.flow_from_directory(TRAIN_DATA_PATH, target_size=(224, 224), batch_size=32, class_mode='categorical')
class_indices = train_generator.class_indices
index_to_class = {v: k for k, v in class_indices.items()}  # Převod indexu na třídu

# Výběr náhodných mincí
coin_folders = [os.path.join(TEST_DATA_PATH, d) for d in os.listdir(TEST_DATA_PATH) if os.path.isdir(os.path.join(TEST_DATA_PATH, d))]
random_coins = random.sample(coin_folders, 10)

# Funkce pro načtení a klasifikaci obrázků
def classify_coin(image_path, model):
    img = load_img(image_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Použití správného preprocessing pro ResNet50
    img_array = preprocess_resnet(img_array)
    
    prediction = model.predict(img_array)
    class_index = np.argmax(prediction)
    class_label = index_to_class[class_index]
    
    return class_label

# Klasifikace vybraných mincí
classified_coins = []
for folder in random_coins:
    coin_images = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(('.jpg', '.png'))]
    if coin_images:
        class_id = classify_coin(coin_images[0], model)
        
        if class_id in cat_to_name:
            coin_info_split = cat_to_name[class_id].split(",")
            
            try:
                # Extrakce číselné hodnoty mince
                value = float(coin_info_split[0].split(" ")[0])  
            except ValueError:
                value = 1.0  # Pokud nelze převést, nastavíme výchozí hodnotu 1.0

            currency_code = coin_info_split[2].strip()  # Použití správného měnového kódu
            
            classified_coins.append({
                "Image": coin_images[0],
                "Class ID": class_id,
                "Coin Info": cat_to_name[class_id],
                "Value": value,
                "Currency Code": currency_code
            })

# Získání aktuálních kurzů z veřejného API ČNB
cnb_api_url = "https://www.cnb.cz/en/financial_markets/foreign_exchange_market/exchange_rate_fixing/daily.txt"
response = requests.get(cnb_api_url)
cnb_data = response.text.splitlines()

# Parsování směnných kurzů
currency_rates = {"CZK": 1.0}  # Základní kurz
for line in cnb_data[2:]:  # Přeskakujeme první dva řádky hlavičky
    parts = line.split("|")
    if len(parts) >= 3:
        currency_code = parts[3].strip()
        exchange_rate = float(parts[4].replace(",", "."))
        currency_rates[currency_code] = exchange_rate

# Přepočet hodnoty mincí do 5 hlavních měn
for coin in classified_coins:
    base_currency = coin["Currency Code"]
    base_rate = currency_rates.get(base_currency, 1)
    
    coin["Value_CZK"] = coin["Value"] * base_rate
    coin["Value_USD"] = coin["Value_CZK"] / currency_rates.get("USD", 1)
    coin["Value_EUR"] = coin["Value_CZK"] / currency_rates.get("EUR", 1)
    coin["Value_JPY"] = coin["Value_CZK"] / currency_rates.get("JPY", 1)
    coin["Value_GBP"] = coin["Value_CZK"] / currency_rates.get("GBP", 1)
    coin["Value_CNY"] = coin["Value_CZK"] / currency_rates.get("CNY", 1)

# Výstupní DataFrame
df = pd.DataFrame(classified_coins)
print(tabulate(df, headers='keys', tablefmt='grid'))


Found 6174 images belonging to 211 classes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
+----+-------------------------------------------------------+------------+--------------------------------------+---------+-----------------+-------------+-------------+-------------+-------------+-------------+-------------+
|    | Image                                                 |   Class ID | Coin Info                            |   Value | Currency Code   |   Value_CZK |   Value_USD |   Value_EUR |   Value_JPY |   Value_GBP |   Value_CNY |
+====+=======================================================+============+======================================+=========